In [16]:
from DATA225utils import make_connection

In [17]:
conn = make_connection(config_file = 'remote.ini')
cursor = conn.cursor()        

In [18]:
cursor.execute("DROP PROCEDURE IF EXISTS SubCategoryInCategoryWithProductsInfo;")
cursor.execute("DROP PROCEDURE IF EXISTS GetOrderHistoryForCustomer;")
cursor.execute("DROP VIEW IF EXISTS CustomerLifetimeValue;")
cursor.execute("DROP VIEW IF EXISTS CustomerLocationOrderFrequency;")

In [19]:
sql = """
    CREATE PROCEDURE SubCategoryInCategoryWithProductsInfo(IN cid INT)
    BEGIN
    SELECT DISTINCT s.Id, s.Name, COUNT(p.ProductId) AS NumOfProducts, ROUND(AVG(p.ProductPrice), 2) AS AvgProductPrice
    FROM Sub_Category s
    JOIN Products p ON s.Id = p.SubCategoryId
    WHERE p.CategoryId = cid
    GROUP BY s.Id, s.Name;
    END
"""
cursor.execute(sql)

In [20]:
sql = """
    CREATE PROCEDURE GetOrderHistoryForCustomer(IN customerId INT)
    BEGIN
        SELECT o.OrderId, o.OrderDate, p.ProductName, od.Quantity, p.ProductPrice, py.Amount
        FROM Orders o
        INNER JOIN Order_Details od ON o.OrderId = od.OrderId
        INNER JOIN Products p ON od.ProductId = p.ProductId
        INNER JOIN Payment py ON o.OrderId = py.OrderId
        WHERE o.CustomerId = customerId;
    END
"""
cursor.execute(sql)

In [21]:
sql = """
    CREATE VIEW CustomerLifetimeValue AS
    SELECT
      Customer.CustomerId,
      Customer.FirstName,
      Customer.LastName,
      Customer.Email,
      SUM(Payment.Amount) AS TotalRevenue,
      COUNT(DISTINCT Orders.OrderId) AS TotalOrders,
      SUM(Payment.Amount) / COUNT(DISTINCT Orders.OrderId) AS AvgRevenuePerOrder
    FROM
      Customer
      INNER JOIN Orders ON Customer.CustomerId = Orders.CustomerId
      INNER JOIN Payment ON Orders.OrderId = Payment.OrderId
    GROUP BY Customer.CustomerId
    ORDER BY TotalRevenue DESC
    LIMIT 1000;
"""
cursor.execute(sql)

In [22]:
sql = """
    CREATE VIEW CustomerLocationOrderFrequency AS
    SELECT
      ca.City,
      ca.State,
      COUNT(o.OrderId) AS OrderFrequency
    FROM
      Customer c
      INNER JOIN Customer_Address ca ON c.CustomerId = ca.CustomerId
      INNER JOIN Orders o ON c.CustomerId = o.CustomerId
    GROUP BY ca.City, ca.State
    ORDER BY OrderFrequency DESC
    LIMIT 300;
"""
cursor.execute(sql)

In [23]:
cursor.close()
conn.close()